### Задание 6. Задача об оптимальном использовании ресурсов

<b>Математическая модель задачи.</b><br>
Необходимо максимизировать функцию<br>
*F(x1,x2,x3) = (6b + 12)\*x1 + (5b + 22)\*x2 + c\*x3* → max<br>
при заданных ограничениях ресурсов:<br>
*x1 + 3\*x2 + a\*x3 <= 3000*<br>
*6\*x1 + 5\*x2 + 2\*x3 <= 3320*<br>
*x1, x2, x3 >= 0*

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
import sympy as sym
import matplotlib.pyplot as plt
from gurobipy import *

In [2]:
a = int(input('a = '))
b = int(input('b = '))
c = int(input('c = '))

a = 4
b = 1
c = 25


#### Оптимальный план производства

linprog

In [3]:
#границы и ограничения
bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])
a_u = np.array([[1, 3, a], [6, 5, 2]])
b_u = np.array([3000, 3320])
c0 = np.array([6*b + 12, 5*b + 22, c])

In [4]:
res = linprog(-c0, a_u, b_u, bounds=bounds, method='simplex', options={'disp': True})

x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
F_max = -res.fun
print(res.x)

Optimization terminated successfully.
         Current function value: -23040.000000
         Iterations: 2
[  0. 520. 360.]


In [5]:
print(f'Оптимальные значения: x1 = {x1.round(5)}, x2 = {x2.round(5)}, x3 = {x3.round(5)}')
print('Максимальное значение:',F_max.round(5))

Оптимальные значения: x1 = 0.0, x2 = 520.0, x3 = 360.0
Максимальное значение: 23040.0


In [6]:
b_optimal = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3]) # Реальный расход ресурсов
b_optimal

array([3000., 3320.])

In [7]:
abs(b_u-b_optimal)

array([0., 0.])

gurobipy

In [9]:
var_s = ['x1', 'x2', 'x3']                         
prof = np.array([6*b + 12, 5*b + 22, c]) # цены
res_used = np.array([[1, 3, a], [6, 5, 2]]) # использование ресурсов
res_av = np.array([3000, 3320]) # запасы

In [10]:
n = len(prof)                                    
m = len(res_av)                                 
model = Model()
x = model.addVars(n, name=var_s) # заносим переменные в модель
obj = model.setObjective(quicksum(prof[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_av[i], name=var_s[i]))
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x29d03107
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+31   3.625000e+30   7.000000e+01      0s
       3    2.3040000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.304000000e+04


In [11]:
# параметры модели
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

X = [0.0, 520.0, 360.0] 
Fmax = 23040.0 
Оптимальный план двойственной задачи Pi → (y1,y2) = [5.071428571428571, 2.357142857142857]
x1 = 0.0
Pi1 = 5.071428571428571


In [12]:
# Оценка устойчивости коэффициентов целевой функции
# Минимальные и максимальные значения коэффициентов
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1            0           18         -inf      19.2143 
          x2          520           27      26.2273         62.5 
          x3          360           25         10.8      26.3077 


In [13]:
#Решение двойственной к M задачи
print('Анализ чувствительности')
print('Fmax =', model.ObjVal)
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])

Анализ чувствительности
Fmax = 23040.0

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1            0     -1.21429            0     -387.692      330.909          inf            0          inf 
          x2          520            0            0         -inf          520          inf          520          inf 
          x3          360            0            0         -inf          360          inf          360          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0      5.07143         3000         1992         6640 
          x2            <            0      2.35714         3320         1500         5000 
